In [1]:
import sys
import os
sys.path.append(os.path.abspath("../")) #Add parent directory

In [2]:
# import torch
# import nibabel as nib
# import os
# import numpy as np

## Load data

In [4]:
from Modular_DL.data_loading import load_nifti, resample_image
from Modular_DL.data_normalization import z_score_normalize
import numpy as np
import SimpleITK as sitk

In [ ]:
folder_path = r"C:\Users\User\Desktop\UDG_old_pc\UDG\Subjects\MISSSSSA\Final_project\TrainingValidationTestSets\Training_Set".replace("\\", "/")


output_folder = r"C:\Users\User\Desktop\UDG_old_pc\UDG\Subjects\MISSSSSA\Final_project\TrainingValidationTestSets\Training_resampled".replace("\\", "/")
os.makedirs(output_folder, exist_ok= True)
#target spacing for now: 1,1,1
target_spacing = (1.0, 1.0, 1.0)
reference_shape = None
reference_affine = None
consistent = True

for root, _, files in os.walk(folder_path):
    for file in files:
        #print(file)
        if file.endswith(".nii") or file.endswith(".nii.gz"):
            file_path = os.path.join(root, file).replace("\\", "/")
            
            if 'seg' in file_path:
                label_data, _, _ = load_nifti(file_path)
                
                label_sitk = sitk.GetImageFromArray(label_data)
                label_sitk.SetSpacing(voxel_spacing)
                label_resample = resample_image(label_sitk, target_spacing, is_label= True)
                label_resample_np = sitk.GetArrayFromImage(label_resample)
                resampled_data = label_resample_np

                #Save resampled label
                output_file_path = os.path.join(output_folder, f"resampled_{file}")
                sitk.WriteImage(label_resample, output_file_path)

                
            else:
                data, affine, voxel_spacing = load_nifti(file_path)

                data = z_score_normalize(data)
                #Resampling data
                image_sitk = sitk.GetImageFromArray(data)
                image_sitk.SetSpacing(voxel_spacing)
                resampled_image = resample_image(image_sitk, target_spacing, is_label=False)
                resampled_image_np = sitk.GetArrayFromImage(resampled_image)
                resampled_data = resampled_image_np

                #Save image
                output_file_path = os.path.join(output_folder, f"resampled_{file}")
                sitk.WriteImage(resample_image, output_file_path)



            if reference_shape is None:
                reference_shape =  resampled_data.shape
                reference_affine = affine
            
            else:

                if resampled_data.shape != reference_shape:
                    print(f"Shape mismatch for file: {file}. Expected {reference_shape}, got {resampled_data.shape}")
                    consistent = False

                if not np.allclose(affine, reference_affine, atol=1e-5):
                    print(f"Affine mismatch for file: {file}")
                    consistent = False
            print("Resampled", resampled_data.shape)


            # print(f"------------------Start {file}--------------------")
            # print(f"File: {file} - Voxel Spacing: {voxel_spacing}")
            # print("\n")
            # print(f"File: {file} - Affine matrix: \n{affine}")
        
            # print(f"------------------End {file}--------------------")
            # print("\n")
            


KeyboardInterrupt: 

In [7]:
resampled_data.shape

(214, 192, 214)